In [2]:
import pandas as pd 


In [3]:
data = pd.read_csv('data.csv')
data.head()

,user_id,date,vertical,turnover_cash_num,turnover_cash_sum,deposit_approved_num,deposit_approved_sum,withdrawal_approved_num,withdrawal_approved_sum,NGR_sum,session_num,session_sum
0,-4233848618748012971,2020-06-19,sports,4,41.662354,0,0.0,0,0.0,59.875489,0,0.0
1,2814934526452133975,2020-09-14,casino_live,123,847.111687,0,0.0,0,0.0,81.685770,0,0.0
2,3718332242005503191,2020-09-06,casino_live,399,21111.275179,0,0.0,0,0.0,2731.476087,0,0.0
3,-5271561423861853461,2020-09-06,casino_live,60,825.512773,0,0.0,0,0.0,-364.196812,0,0.0
4,-3697770787649913393,2020-07-25,casino_live,13,105.128915,0,0.0,0,0.0,105.128915,0,0.0


In [3]:
data.describe()

,user_id,turnover_cash_num,turnover_cash_sum,deposit_approved_num,deposit_approved_sum,withdrawal_approved_num,withdrawal_approved_sum,NGR_sum,session_num,session_sum
count,1.582160e+05,158216.000000,1.582160e+05,158216.000000,158216.000000,158216.000000,158216.000000,158216.000000,158216.000000,158216.000000
mean,1.503073e+17,517.183490,6.965408e+02,3.052845,110.030905,0.588164,89.435863,26.190897,4.512578,8461.784323
std,5.183782e+18,1425.576709,7.700157e+03,6.477548,481.580646,2.006541,1690.318934,1492.255301,7.462878,16237.851636
min,-9.221952e+18,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-432169.795773,0.000000,0.000000
25%,-4.287080e+18,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.441136e+17,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1039.867168
75%,4.478151e+18,290.000000,2.713875e+02,4.000000,61.248321,0.000000,0.000000,42.705519,7.000000,9572.432815
max,9.221475e+18,36798.000000,2.420897e+06,180.000000,60143.763871,66.000000,557180.923178,309173.160776,154.000000,662316.846415


In [6]:
data.columns

Index(['user_id', 'date', 'vertical', 'turnover_cash_num', 'turnover_cash_sum',
       'deposit_approved_num', 'deposit_approved_sum',
       'withdrawal_approved_num', 'withdrawal_approved_sum', 'NGR_sum',
       'session_num', 'session_sum'],
      dtype='object')

# Data preprocessing

## missing

In [7]:
data.isnull().sum()

user_id                        0
date                           0
vertical                   82659
turnover_cash_num              0
turnover_cash_sum              0
deposit_approved_num           0
deposit_approved_sum           0
withdrawal_approved_num        0
withdrawal_approved_sum        0
NGR_sum                        0
session_num                    0
session_sum                    0
dtype: int64

## outlier

In [12]:
# Convert relevant columns to numeric
columns_to_convert = ['turnover_cash_num', 'turnover_cash_sum', 'deposit_approved_num', 'deposit_approved_sum',
                      'withdrawal_approved_num', 'withdrawal_approved_sum', 'NGR_sum', 'session_num', 'session_sum']

for column in columns_to_convert:
    data[column] = pd.to_numeric(data[column], errors='coerce')

q1 = data[columns_to_convert].quantile(.25)
q3 = data[columns_to_convert].quantile(.75)
iqr =q3-q1

outliers = ((data[columns_to_convert] < (q1 - 1.5 * iqr)) | (data[columns_to_convert] > (q3 +1.5 * iqr))).sum
outliers

<bound method DataFrame.sum of         turnover_cash_num  turnover_cash_sum  deposit_approved_num  \
0                   False              False                 False   
1                   False               True                 False   
2                   False               True                 False   
3                   False               True                 False   
4                   False              False                 False   
...                   ...                ...                   ...   
158211              False              False                 False   
158212               True               True                 False   
158213              False              False                 False   
158214              False              False                 False   
158215              False              False                 False   

        deposit_approved_sum  withdrawal_approved_num  \
0                      False                    False   
1             

## one-hot 

In [13]:
data = pd.get_dummies(data, columns=['vertical'])

In [14]:
data

,user_id,date,turnover_cash_num,turnover_cash_sum,deposit_approved_num,deposit_approved_sum,withdrawal_approved_num,withdrawal_approved_sum,NGR_sum,session_num,session_sum,vertical_bingo,vertical_casino_classic,vertical_casino_live,vertical_sports
0,-4233848618748012971,2020-06-19,4,41.662354,0,0.0,0,0.0,59.875489,0,0.0,False,False,False,True
1,2814934526452133975,2020-09-14,123,847.111687,0,0.0,0,0.0,81.685770,0,0.0,False,False,True,False
2,3718332242005503191,2020-09-06,399,21111.275179,0,0.0,0,0.0,2731.476087,0,0.0,False,False,True,False
3,-5271561423861853461,2020-09-06,60,825.512773,0,0.0,0,0.0,-364.196812,0,0.0,False,False,True,False
4,-3697770787649913393,2020-07-25,13,105.128915,0,0.0,0,0.0,105.128915,0,0.0,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158211,-7117022959737387082,2020-06-20,158,216.515085,0,0.0,0,0.0,2.965960,0,0.0,False,False,True,False
158212,2811336782031493717,2020-03-12,899,1951.778557,0,0.0,0,0.0,567.065974,0,0.0,False,True,False,False
158213,-3328374110339852023,2020-02-27,0,0.000000,0,0.0,0,0.0,2.370862,0,0.0,False,False,True,False
158214,3047797546794493486,2020-03-25,4,0.287459,0,0.0,0,0.0,0.287459,0,0.0,False,False,True,False


# Feature engineering

- we will create some aditional features for this model

In [15]:
data['avg_turnover'] = data['turnover_cash_sum'] / data['turnover_cash_num']
data['total_transactions'] = data['deposit_approved_num'] + data['withdrawal_approved_num']
data['date'] = pd.to_datetime(data['date'])
data['recency'] = (data['date'].max() - data['date']).dt.days
data['frequency'] = data['session_num']
data['monetary_value'] = data['NGR_sum']


# Model Selection and Training


In [25]:
# Aggregate the data to get the most recent date per user
recent_date_per_user = data.groupby('user_id')['date'].max().reset_index()

# Merge this back to the original data
data = data.merge(recent_date_per_user, on='user_id', suffixes=('', '_most_recent'))

# Define churn as users with no sessions or transactions in the last 30 days
data['churn_flag'] = (data['date_most_recent'].max() - data['date_most_recent']).dt.days > 30


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Exclude datetime columns
X = data.drop(columns=['user_id', 'date', 'date_most_recent', 'churn_flag'])
y = data['churn_flag']



In [41]:

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Fit and transform the data
X_imputed = imputer.fit_transform(X)

# Convert back to DataFrame if necessary
X = pd.DataFrame(X_imputed, columns=X.columns)


# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [42]:
import numpy as np

# Replace inf with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Check for excessively large values (e.g., larger than a threshold like 1e10)
threshold = 1e10
X[X > threshold] = np.nan

# Now reapply the imputer
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed, columns=X.columns)


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the data again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Accuracy: 0.6633940798483092
Precision: 0.4365748121156912
Recall: 0.4019991611911086
F1 Score: 0.4185741839222679


c:\Users\Andre\Documents\GitHub\Projects\venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- let's try to scale the values for the Logistic model

In [45]:
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model again
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Accuracy: 0.7252291161908775
Precision: 0.5745986779981114
Recall: 0.3402768069341535
F1 Score: 0.42742997629291424


In [44]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f'Random Forest - Accuracy: {accuracy_rf}')
print(f'Random Forest - Precision: {precision_rf}')
print(f'Random Forest - Recall: {recall_rf}')
print(f'Random Forest - F1 Score: {f1_rf}')


Random Forest - Accuracy: 0.7154745602022543
Random Forest - Precision: 0.5398309298856291
Random Forest - Recall: 0.37942122186495175
Random Forest - F1 Score: 0.44563031074258036


## Hyperparameter tuning

In [46]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Perform grid search
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters
print(grid_search.best_params_)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


c:\Users\Andre\Documents\GitHub\Projects\venv\lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 200}


In [48]:
# Initialize the Random Forest with the best parameters
optimized_rf_model = RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200, random_state=42)

# Train the optimized model
optimized_rf_model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred_optimized_rf = optimized_rf_model.predict(X_test)
accuracy_optimized_rf = accuracy_score(y_test, y_pred_optimized_rf)
precision_optimized_rf = precision_score(y_test, y_pred_optimized_rf)
recall_optimized_rf = recall_score(y_test, y_pred_optimized_rf)
f1_optimized_rf = f1_score(y_test, y_pred_optimized_rf)

print(f'Optimized Random Forest - Accuracy: {accuracy_optimized_rf}')
print(f'Optimized Random Forest - Precision: {precision_optimized_rf}')
print(f'Optimized Random Forest - Recall: {recall_optimized_rf}')
print(f'Optimized Random Forest - F1 Score: {f1_optimized_rf}')


Optimized Random Forest - Accuracy: 0.7347940587801538
Optimized Random Forest - Precision: 0.6207138842046094
Optimized Random Forest - Recall: 0.3087515727666713
Optimized Random Forest - F1 Score: 0.41237979647091777
